In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Binarizer

from sklearn.pipeline import Pipeline

from feature_engine.imputation import(
    AddMissingIndicator,
    MeanMedianImputer,
    CategoricalImputer
)

from feature_engine.encoding import (
    RareLabelEncoder,
    OrdinalEncoder
)

from feature_engine.transformation import LogTransformer

from feature_engine.selection import DropFeatures
from feature_engine.wrappers import SklearnTransformerWrapper

import joblib

from keras.models import Model
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasRegressor
from keras.activations import relu, sigmoid
from keras.layers import Dense, Activation
from keras.models import Sequential

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.metrics import mean_squared_error

In [2]:
import my_preprocessors as mypp #nuestra libreria

In [3]:
data = pd.read_csv('marketing_campaign.csv', sep='\t')
data

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,13-06-2013,46,709,...,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,10-06-2014,56,406,...,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,25-01-2014,91,908,...,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,24-01-2014,8,428,...,3,0,0,0,0,0,0,3,11,0


In [4]:
data[['dia', 'mes', 'Year']] = data['Dt_Customer'].str.split('-', n=2, expand=True)

In [5]:
data.shape

(2240, 32)

In [6]:
data["dia"] = pd.to_numeric(data["dia"])
data["mes"] = pd.to_numeric(data["mes"])
data["Year"] = pd.to_numeric(data["Year"])
del(data['Dt_Customer'])

In [7]:
data.shape

(2240, 31)

In [8]:
type(data["dia"])

pandas.core.series.Series

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['NumWebPurchases'], axis=1),
        data['NumWebPurchases'],
        test_size=0.1,
        random_state=2022)

X_train.shape, X_test.shape

((2016, 30), (224, 30))

## Configuración del Machine Learning Pipeline

In [10]:
#Variables categoricas con NA
CATEGORICAL_VARS_WITH_NA_FREQUENT = []

#Variable categoricas con NA pero indicador de Missing
CATEGORICAL_VARS_WITH_NA_MISSING = []

#Variables numéricas con NA
NUMERICAL_VARS_WITH_NA = ['Income']

#Variables de temporalidad
TEMPORAL_VARS = ['Year_Birth']

REF_VAR = "Year"

#Variable que se elimina
DROP_FEATURES = ['Year']

#Varibles para transformación logaritmia
#NUMERICALS_LOG_VARS = []

#Variables para binarización por sesgo fuerte
BINARIZE_VARS = ['MntFruits', 'MntSweetProducts', 'MntFishProducts', 'MntMeatProducts','MntWines','MntGoldProds']

#Variables para hacer mapeo categorico por codificación ordinal
QUAL_VARS = []

EXPOSURE_VARS = []

FINISH_VARS = []

GARAGE_VARS = []

FENCE_VARS = []

#Variables categoricas a codificar sin ordinalidad
CATEGORICAL_VARS = ['Education', 'Marital_Status']

#Mapeos de variables categoricas
QUAL_MAPPINGS = {}

EXPOSURE_MAPPINGS = {}

FINISH_MAPPINGS = {}

GARAGE_MAPPINGS = {}

#Variables seleccionadas según análisis de Lasso
FEATURES = [
    'Education', 
    'Marital_Status',
    'ID',
    'Year_Birth',
    'Income',
    'Kidhome',
    'Teenhome',
    'Recency',
    'MntWines',
    'MntFruits',
    'MntMeatProducts',
    'MntFishProducts',
    'MntSweetProducts',
    'MntGoldProds',
    'NumDealsPurchases',
    'NumCatalogPurchases',
    'NumStorePurchases',
    'NumWebVisitsMonth',
    'AcceptedCmp3',
    'AcceptedCmp4',
    'AcceptedCmp5',
    'AcceptedCmp1',
    'AcceptedCmp2',
    'Complain',
    'Z_CostContact',
    'Z_Revenue',
    'Response',
    'dia',
    'mes',
    'Year',
]

In [11]:
#Selección de variables para entrenamiento
X_train = X_train[FEATURES]

In [12]:
#Seleccionamos variables para predicción
X_test = X_test[FEATURES]

In [13]:
def create_model():
    clasificador2 = Sequential()
    clasificador2.add(Dense(input_dim=30, kernel_initializer="he_uniform", units=16, activation='relu'))
    clasificador2.add(Dense(kernel_initializer="he_uniform", units=11, activation='relu'))
    clasificador2.add(Dense(units=1, kernel_initializer="he_uniform", activation="linear"))
    # Parametros de entrenamiento.
    clasificador2.compile(optimizer='adam', loss='mean_squared_error', metrics=['MSE'])

    return clasificador2

# wrap the model using the function you created
clf = KerasRegressor(build_fn=create_model,verbose=0)

#clasificador2.fit(X_train, y_train, batch_size=15, epochs=100)

## Machine Learning PipeLine

In [14]:
WebPurchase_pipeline = Pipeline([
    
    #============= IMPUTACIONES ===================#
    
    #1. Imputación de variables categoricas
    ('missing_imputation', 
         CategoricalImputer(imputation_method='missing', variables=CATEGORICAL_VARS_WITH_NA_MISSING)
    ),
    
    #2. Imputación de variables categoricas con NA basado en frecuencia.
    ('frequent_imputation', 
         CategoricalImputer(imputation_method='frequent', variables=CATEGORICAL_VARS_WITH_NA_FREQUENT)
    ),
    
    #3. Indicamos Faltante en variables numéricas para imputar
    ('missing_indicator', AddMissingIndicator(variables=NUMERICAL_VARS_WITH_NA)),
    
    #4. Imputación de mediana para variables categoricas
    ('mean_imputation', MeanMedianImputer(
        imputation_method='mean', variables=NUMERICAL_VARS_WITH_NA)
    ),
    
    #============= VARIABLES TEMPORALES ==================
    
    #5. Tratamiento de variables temporales
    ('eslapsed_time', mypp.TemporalVariableTransformer(
        variables=TEMPORAL_VARS, reference_variable=REF_VAR)
    ),
    
    #6. Drop de variables
    ('drop_features', DropFeatures(features_to_drop=DROP_FEATURES)),
    
    #============= TRANSFORMACIÓN DE VARIABLES NUMÉRICAS =============
    
    #7. Transformación logaritmica
    #('log', LogTransformer(variables=NUMERICALS_LOG_VARS)), no aplica en el dataset utilizado para proyecto.
    
    #8. Binarización de Variables con Sesgo Fuerte
    ('binarizer', SklearnTransformerWrapper(
        transformer=Binarizer(threshold=0), variables=BINARIZE_VARS)
    ),
    
    #=============== CODIFICACION DE VARIABLES CATEGORICAS ORDINALES ==============
    #no se tienen en el dataset utilizado para proyecto.
    
    #============ CODIFICACION DE VARIABLES CATEGORICAS NOMINALES ============
    #('Clasificacion', Sequential.compile(optimizer='adam', loss='mean_squared_error', metrics=['MSE'])),
    # En mi neurona está clasificador.fit(X_train, y_train, batch_size=15, epochs=100)  
    
    ('rare_label_encoder', RareLabelEncoder(
        tol=0.01, n_categories=1, variables=CATEGORICAL_VARS)),
    
    ('categorical_encoder', OrdinalEncoder(
        encoding_method='ordered', variables=CATEGORICAL_VARS)),
    
    #=========== SCALER ==============
    ('scaler', MinMaxScaler()),
    
    #=========== ENTRENAMIENTO DEL MODELO ============
    ('clf',clf),

]) 

In [15]:
WebPurchase_pipeline.fit(X_train, y_train)

Pipeline(steps=[('missing_imputation', CategoricalImputer(variables=[])),
                ('frequent_imputation',
                 CategoricalImputer(imputation_method='frequent',
                                    variables=[])),
                ('missing_indicator',
                 AddMissingIndicator(variables=['Income'])),
                ('mean_imputation',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['Income'])),
                ('eslapsed_time',
                 TemporalVariableTransforme...
                                                      'MntFishProducts',
                                                      'MntMeatProducts',
                                                      'MntWines',
                                                      'MntGoldProds'])),
                ('rare_label_encoder',
                 RareLabelEncoder(n_categories=1, tol=0.01,
                                  variables=['Ed

In [16]:
preds = WebPurchase_pipeline.predict(X_test)

In [17]:
mean_squared_error(np.exp(y_test), np.exp(preds), squared=False)

35548968384.44217

In [18]:
r2_score(y_test, preds)

-0.2094318414524754

In [19]:
y_test

616     1
2133    2
2117    6
1149    1
596     2
       ..
1147    9
567     1
89      2
1522    2
2215    1
Name: NumWebPurchases, Length: 224, dtype: int64

In [20]:
preds

array([1.2815584 , 2.305147  , 2.8364258 , 1.895495  , 2.7123144 ,
       2.369794  , 1.9772053 , 2.9227931 , 2.5040634 , 2.4754174 ,
       2.904753  , 2.2315466 , 2.9006639 , 2.6176615 , 2.8829725 ,
       2.5279486 , 2.775851  , 1.7184129 , 3.2642043 , 2.2297661 ,
       1.5788682 , 2.202609  , 2.3454094 , 2.6204545 , 2.6350515 ,
       3.0659425 , 2.1436467 , 2.3047442 , 3.3708398 , 2.4705317 ,
       2.8346725 , 2.7865777 , 1.9779981 , 2.5812633 , 2.4020205 ,
       2.641595  , 2.9010098 , 2.4093707 , 2.7081964 , 2.0642288 ,
       1.9665515 , 2.6410315 , 2.603507  , 2.532975  , 2.724782  ,
       1.8518912 , 1.9081168 , 2.063933  , 1.9806503 , 1.9384604 ,
       1.4914339 , 2.8166687 , 2.690277  , 2.3314943 , 1.7883773 ,
       2.5751393 , 2.5108163 , 1.7136006 , 2.9002492 , 2.8865995 ,
       1.7016698 , 2.8201644 , 2.0799391 , 1.2882806 , 2.8639777 ,
       2.915778  , 2.9434137 , 2.9428265 , 2.8079422 , 2.6713784 ,
       1.9393682 , 2.8059266 , 2.7528164 , 2.7404459 , 2.45312

In [21]:
import joblib
#Guardamos pipeline, OJO modificar ruta para realizar esta parte sin inconvenientes.
joblib.dump(r"C:\Users\kimbe\OneDrive - Universidad Galileo\Octavo Trimestre\Product Development\Proyecto 1 Kimberly Rivera\WebPurchase_pipeline", r"C:\Users\kimbe\OneDrive - Universidad Galileo\Octavo Trimestre\Product Development\Proyecto 1 Kimberly Rivera\WebPurchase_pipeline.pkl")

['C:\\Users\\kimbe\\OneDrive - Universidad Galileo\\Octavo Trimestre\\Product Development\\Proyecto 1 Kimberly Rivera\\WebPurchase_pipeline.pkl']

In [22]:
type(WebPurchase_pipeline)

sklearn.pipeline.Pipeline